In [221]:
###################### Importar Librerias ######################
import numpy as np
import pandas as pd

In [222]:
###################### Funcion calcular sindrome ######################
def calcular_sindrome(patron_error, matriz_Ht):
    sindrome = []
    detectable = []
    for i in patron_error:
        valor = []
        i2 = [np.float64(j) for j in np.array(list(i))]
        for x in range(len(matriz_Ht[0])):
            multiply_ih = i2*matriz_Ht[:,x]
            valid = np.count_nonzero(multiply_ih == 1)
            if valid%2 != 0:
                valor.append(1)
            else:
                valor.append(0)
        sindrome.append(valor)
        if valor == [0, 0, 0]:
            detectable.append("no")
        else:
            detectable.append("si")
    return sindrome, detectable
###################### Funcion para la lectura del archivo de matlab ###################### 
def leer_mensaje(mensaje_llegada,barker):
    mensaje_respuesta = ''
    for i in mensaje_llegada:
        if i != ',':
            if i != '\n':
                mensaje_respuesta+=i
    mensaje_respuesta = mensaje_respuesta[mensaje_respuesta.find(barker)+len(barker):]
    return mensaje_respuesta

In [223]:
###################### Datos para el ejercicio ######################
n=7
k=4
p_e = 1e-3
matriz_P = np.array([[1,1,0],[0,1,1],[1,1,1],[1,0,1]], dtype=int)
matriz_Ik = np.identity(k,dtype=int)
matriz_G = np.concatenate((matriz_P.T,matriz_Ik.T),axis=0,dtype=int).T
matriz_H = np.concatenate((matriz_Ik[:-1].T[:-1],matriz_P),axis=0,dtype=int).T
matriz_Ht = matriz_H.T
barker = "1111100110101"
ruido = "00000000000000000000000"

In [224]:
## calculo de patrones de error
patron_error = np.array(["{0:07b}".format(x) for x in range(128)])
cantidad_error = np.array([x.count('1') for x in patron_error])
sindrome, detectable = calcular_sindrome(patron_error,matriz_Ht)
corregibles = list(map(lambda x: "no" if x>1 else "si", cantidad_error))
probabilidad = [p_e**x * (1-p_e)**(n-x) for x in cantidad_error]

In [225]:
calculo_patrones = pd.DataFrame({'patron_error':patron_error,'cantidad_error':cantidad_error,'sindrome':sindrome,'detectable':detectable,'corregibles':corregibles,'probabilidad':probabilidad})
calculo_patrones

,patron_error,cantidad_error,sindrome,detectable,corregibles,probabilidad
0,0000000,0,"[0, 0, 0]",no,si,9.930210e-01
1,0000001,1,"[1, 0, 1]",si,si,9.940150e-04
2,0000010,1,"[1, 1, 1]",si,si,9.940150e-04
3,0000011,2,"[0, 1, 0]",si,no,9.950100e-07
4,0000100,1,"[0, 1, 1]",si,si,9.940150e-04
...,...,...,...,...,...,...
123,1111011,6,"[0, 1, 1]",si,no,9.990000e-19
124,1111100,5,"[0, 1, 0]",si,no,9.980010e-16
125,1111101,6,"[1, 1, 1]",si,no,9.990000e-19
126,1111110,6,"[1, 0, 1]",si,no,9.990000e-19


In [226]:
###################### Sindrome ######################
lista_errores_corregibles = []
lista_sindrome = []
for indice in range(len(calculo_patrones)):
    if (calculo_patrones.patron_error[indice].count('1') < 2):
        lista_errores_corregibles.append(calculo_patrones.patron_error[indice])
        lista_sindrome.append(calculo_patrones.sindrome[indice])
tabla_sindrome = pd.DataFrame({'patron_error_corregible':lista_errores_corregibles,'sindrome':lista_sindrome})
tabla_sindrome

,patron_error_corregible,sindrome
0,0000000,"[0, 0, 0]"
1,0000001,"[1, 0, 1]"
2,0000010,"[1, 1, 1]"
3,0000100,"[0, 1, 1]"
4,0001000,"[1, 1, 0]"
5,0010000,"[0, 0, 1]"
6,0100000,"[0, 1, 0]"
7,1000000,"[1, 0, 0]"


In [227]:
###################### Transmisor ######################
f = open ('mensaje_codificado.txt','r')
mensaje = f.read()
while(len(mensaje)%k != 0):
    mensaje+='0'
mensaje = np.array(list(map(list, zip(*[map(int, list(mensaje))] * k))),dtype=int)
mensaje = mensaje @ matriz_G
mensaje_generadora = [[0 if a%2==0 else 1 for a in x] for x in mensaje]
mensaje_enviar = ruido+barker+"".join(["".join([str(ele) for ele in sub]) for sub in mensaje_generadora])
print(mensaje_enviar)
f = open ('mensaje_transmisor.txt','w')
f.write(mensaje_enviar)
f.close()

0000000000000000000000011111001101011110010101000110100010000000110010111100100010111000110100011010010111110100000011010010111011010010100010100011000110111111110100011110010111010000110100101000101110011011100000000000011011100101011100110001101001011110100010100011000110011100101110010010111000000000011010111001111111111001010000000011010011001011000110111001001000111100101010111001000111011100101110001110010010111100011010111000100011110010111001010111001101000101000110000000001011111010001101000110100011100100000000000000011010000110100000000001110010010111110010100000001000110110010100000001011100010001100000001110010011010011100101010001000110110010110000000111001011001011101000110100001110011100101010001111100100010111011010010100010000000101000100110101000110110010110111000011010000000010100010110100000000010001101100101101000100000001010001010001100000000001101001101000101111101000101000101000111011100100011001101000010111110100000011010000000100011000000000000000011010011010

In [228]:
###################### Receptor ######################
f = open ('bits_salida.txt','r')
mensaje = f.read()
f.close()
mensaje = leer_mensaje(mensaje,barker)
while(len(mensaje)%n != 0):
    mensaje+='0'
mensaje = np.array(list(map(list, zip(*[map(int, list(mensaje))] * (n)))),dtype=int)
lista_mensaje = ["".join(item) for item in mensaje.astype(str)]
cantidad_error_recepcion = np.array([x.count('1') for x in lista_mensaje])
sindrome_recepcion, detectable_recepcion = calcular_sindrome(lista_mensaje,matriz_Ht)
lista_sindrome = list(tabla_sindrome.sindrome)
corregibles_recepcion = list(map(lambda x: "si" if lista_sindrome else "no", sindrome_recepcion))
probabilidad_recepcion = [p_e**x * (1-p_e)**(n-x) for x in cantidad_error_recepcion]
calculo_patrones_recepcion = pd.DataFrame({'mensaje_recibido':lista_mensaje,'numero_unos':cantidad_error_recepcion,'sindrome':sindrome_recepcion,'detectable':detectable_recepcion,'corregibles':corregibles_recepcion,'probabilidad':probabilidad_recepcion})
calculo_patrones_recepcion

,mensaje_recibido,numero_unos,sindrome,detectable,corregibles,probabilidad
0,1110010,4,"[0, 0, 0]",no,si,9.970030e-13
1,1010001,3,"[0, 0, 0]",no,si,9.960060e-10
2,1010001,3,"[0, 0, 0]",no,si,9.960060e-10
3,0000000,0,"[0, 0, 0]",no,si,9.930210e-01
4,1100101,4,"[0, 0, 0]",no,si,9.970030e-13
...,...,...,...,...,...,...
164,0011010,3,"[0, 0, 0]",no,si,9.960060e-10
165,1010001,3,"[0, 0, 0]",no,si,9.960060e-10
166,1010100,3,"[1, 1, 0]",si,si,9.960060e-10
167,0000101,2,"[1, 1, 0]",si,si,9.950100e-07


In [229]:
## Correjir mensaje
lista_correjibles = list(np.where(calculo_patrones_recepcion.corregibles=="si")[0])
for i in lista_correjibles:
    sindrome_valor = calculo_patrones_recepcion.sindrome.iloc[i]
    mensaje_a_corregir = calculo_patrones_recepcion.mensaje_recibido.iloc[i]
    patron_mensaje_error = tabla_sindrome.patron_error_corregible.iloc[list(tabla_sindrome.sindrome).index(sindrome_valor)]
    sum = "{0:07b}".format(int(mensaje_a_corregir, 2) + int(patron_mensaje_error, 2))
    calculo_patrones_recepcion.mensaje_recibido.iloc[i] = sum
calculo_patrones_recepcion

/tmp/ipykernel_31771/1321343822.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calculo_patrones_recepcion.mensaje_recibido.iloc[i] = sum


,mensaje_recibido,numero_unos,sindrome,detectable,corregibles,probabilidad
0,1110010,4,"[0, 0, 0]",no,si,9.970030e-13
1,1010001,3,"[0, 0, 0]",no,si,9.960060e-10
2,1010001,3,"[0, 0, 0]",no,si,9.960060e-10
3,0000000,0,"[0, 0, 0]",no,si,9.930210e-01
4,1100101,4,"[0, 0, 0]",no,si,9.970030e-13
...,...,...,...,...,...,...
164,0011010,3,"[0, 0, 0]",no,si,9.960060e-10
165,1010001,3,"[0, 0, 0]",no,si,9.960060e-10
166,1011100,3,"[1, 1, 0]",si,si,9.960060e-10
167,0001101,2,"[1, 1, 0]",si,si,9.950100e-07


In [230]:
## Mensaje a decodificar
mensaje_a_decodificar= "".join([x[3:] for x in calculo_patrones_recepcion.mensaje_recibido])
f = open ('mensaje_a_decodificar.txt','w')
f.write(mensaje_a_decodificar)
f.close()
mensaje_a_decodificar

'0010000100010000010100100111110111010111100011010111010000010011110111110011010110000100000110011100000011010101100101101011100000010110100110010111000011011001111101010000010001010110001000110101111000111100110010010111011011000011010101011001000100110000011110001000100000100000000010000100000010010111010100000110010100001100001100000010010000100001110110110000001001011000100010010101001100100111010000010000000110100110010111001010000000010100000001100101000100000001001100001101101001111000000100111100011001000111100011010000011000000000010010001101001110000100010001111010100010000000101000000101100110001000000001000110010110000100101111000111010010100001110011010000'